Deep Learning
=============

Assignment 5
------------

The goal of this assignment is to train a Word2Vec skip-gram model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

Download the data from the source website if necessary.

In [0]:
url = 'http://mattmahoney.net/dc/'
data_root = './data'
def maybe_download(fname, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  dst_fname = os.path.join(data_root, fname)
  if not os.path.exists(dst_fname):
    dst_fname, _ = urlretrieve(url + fname, dst_fname)
  statinfo = os.stat(dst_fname)
  if statinfo.st_size == expected_bytes:
    print('Found/verified %s: Stored it in %s' % (fname, dst_fname))
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + fname + '. Can you get to it with a browser?')
  return dst_fname

filename = maybe_download('text8.zip', 31344016)

Read the data into a string.

In [0]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

Build the dictionary and replace rare words with UNK token.

In [0]:
vocabulary_size = 50000

def build_dataset(words):
  count = [('UNK', -1)]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0] = ('UNK', unk_count)
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

In [0]:
print("# words=%d"%len(data))

Function to generate a training batch for the skip-gram model.

In [0]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

Train a skip-gram model.

In [0]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [0]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

In [0]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [0]:
def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  pylab.figure(figsize=(15,15))  # in inches
  for i, label in enumerate(labels):
    x, y = embeddings[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
  pylab.show()

words = [reverse_dictionary[i] for i in range(1, num_points+1)]
plot(two_d_embeddings, words)

---

Problem
-------

An alternative to skip-gram is another Word2Vec model called [CBOW](http://arxiv.org/abs/1301.3781) (Continuous Bag of Words). In the CBOW model, instead of predicting a context word from a word vector, you predict a word from the sum of all the word vectors in its context. Implement and evaluate a CBOW model trained on the text8 dataset.

---

In [0]:
def gen_skip_batch(self):
  return generate_batch(self.batch_size, 2, 1)

In [0]:
def gen_cbow_batch(self):
  n_win_words = 2*self.cbow_win
  max_index = len(data) - n_win_words
  assert self.word_index < max_index
  batch = np.ndarray(shape=(self.batch_size, n_win_words), dtype=np.int32)
  labels = np.ndarray(shape=(self.batch_size, 1), dtype=np.int32)
  for i in range(self.batch_size):
    labels[i] = data[self.word_index + self.cbow_win]
    for j in range(self.cbow_win):
      batch[i][j] = data[self.word_index + j]
      batch[i][j + self.cbow_win] = data[self.word_index + j + self.cbow_win + 1]
    self.word_index = (self.word_index + 1) % max_index
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:16]])

class CB():
  word_index = 0
  gen_cbow_batch = gen_cbow_batch
  def __init__(self, batch_size, cbow_win):
    self.batch_size = batch_size
    self.cbow_win   = cbow_win
  
    
for cbow_window in [1, 2]:
  cb = CB(batch_size=8, cbow_win=cbow_window)
  batch, labels = cb.gen_cbow_batch()
  print('\nwith cbow_window = %d:' % cbow_window)
  print('  1. (batch, label):', [([reverse_dictionary[wi] for wi in bi], reverse_dictionary[la]) for bi, la in zip(batch, labels.reshape(-1))])
  batch, labels = cb.gen_cbow_batch()
  print('  2. (batch, label):', [([reverse_dictionary[wi] for wi in bi], reverse_dictionary[la]) for bi, la in zip(batch, labels.reshape(-1))])

In [0]:
# WORD 2 VEC Model (CBOW)
# Softmax Score: Input => Avg(Embedding) => Projection(FC) => SampledSofmax
def define_state_cbow(self):
  with self.graph.as_default():
    # Input Layer placeholders for data and labels
    self.dataset_ph = tf.placeholder(tf.int32,
                                    shape=[None, 2*self.cbow_win], name="dataset")
    self.labels_ph = tf.placeholder(tf.int32,
                                    shape=[None, 1], name="labels")
    self.dropout_ph = tf.placeholder(tf.float32, name= "dropout")
    self.l2_reg_ph = tf.placeholder(tf.float32, name= "l2_reg")
    self.wordset_ph = tf.placeholder(tf.int32,
                                    shape=[None], name="wordset")

    # Embedding Layer:
    # Bound the weights to prevent exploding gradient problem
    # Embedding: Use random uniform initializer
    uniform_init = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope("Embedding", initializer=uniform_init) as scope:
      embeds = tf.get_variable("Embeds",
                               [vocabulary_size, self.embedding_size], initializer=uniform_init)

    # Softmax Score Layer: 
    # Softmax Weights: Use truncated normal initializer
    tr_norm_init = tf.truncated_normal_initializer(stddev=1.0/math.sqrt(self.embedding_size))
    # Softmax Biases: Use zeros initializer
    with tf.variable_scope("Softmax", initializer=tr_norm_init) as scope:
      sm_weights = tf.get_variable("Weights", [vocabulary_size, self.embedding_size])
      sm_biases    = tf.get_variable("Biases", [vocabulary_size], initializer=tf.zeros_initializer)

  return

In [0]:
# WORD 2 VEC Model (CBOW)
# Softmax Score: Input => Avg(Embedding) => Projection(FC) => SampledSofmax
def define_computation_cbow(self):
  with self.graph.as_default():
    # Embedding
    with tf.variable_scope("Embedding", reuse=True) as scope:
      embeds = tf.get_variable("Embeds")
    # win_embeds: Dim(win_embeds): [None, 2*cbow_win, d]
    win_embeds  = tf.nn.embedding_lookup(embeds, self.dataset_ph)
    # embed_list: [e1, e2, ..., en] of length 2*cbow_win, where Dim(ei) = [None, d]
    embed_list = tf.unstack(win_embeds, axis=1)
    # CBOW
    avg_embed = tf.scalar_mul(1.0/(2*self.cbow_win), tf.add_n(embed_list))
    
    # SoftmaxScore
    with tf.variable_scope("Softmax", reuse=True) as scope:
      sm_weights = tf.get_variable("Weights")
      sm_biases    = tf.get_variable("Biases")
    sm_loss = tf.nn.l2_loss(sm_weights)
      
    # Cross Entropy Loss
    XEnt = tf.nn.sampled_softmax_loss(weights=sm_weights, biases=sm_biases,
                                      inputs=avg_embed, labels=self.labels_ph,
                                      num_sampled = self.n_samples, num_classes=vocabulary_size)
  
    # Scalar Loss
    self.loss_op = tf.reduce_mean(XEnt) + self.l2_reg_ph*sm_loss
  
    # Optimizer with exponential decay of learning rate
    self.train_op = tf.train.AdagradOptimizer(self.lr).minimize(self.loss_op)

    # Validation: Compute similarity between minibatch and vocabulary
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeds),axis=1,keep_dims=True))
    self.embeds_op = embeds/norm
    valid_embeds = tf.nn.embedding_lookup(self.embeds_op, self.wordset_ph)
    sim_mat = tf.matmul(valid_embeds, tf.transpose(self.embeds_op))
    # Store the indices of the top_k similar words
    _, self.top_k_op = tf.nn.top_k(sim_mat, k=self.valid_top_k, sorted=True)
    
  return

In [0]:
def run_epochs_cbow(self):
  with tf.Session(graph=self.graph) as s:
    initer = tf.global_variables_initializer()
    s.run(initer)
    print("Initialized")
    tot_loss = 0; num_loss = 0
    for step in range(self.n_steps):
      # Generate a minibatch.
      batch_data, batch_labels = self.get_data()
      # Dictionary: Feeds Training Minibatch.
      feed_dict = {
        self.dataset_ph:    batch_data,
        self.labels_ph:       batch_labels,
        self.l2_reg_ph:       self.l2_reg,
        self.dropout_ph:   self.dropout
      }
      _, tr_loss = s.run( [self.train_op, self.loss_op], feed_dict=feed_dict)
      tot_loss += tr_loss; num_loss += 1

      # Compute average loss every n_avg steps
      if (step % self.n_avg == 0):
        print('Step %d: Average Loss=%f' % (step, tot_loss/num_loss))
        tot_loss = 0; num_loss = 0

      # Compute validation accuracy every n_validate steps
      if (step % self.n_validate == 0):  
        val_dict = {self.wordset_ph: self.valid_examples}
        top_k = s.run(self.top_k_op, feed_dict=val_dict)
        valid_words = [reverse_dictionary[i] for i in self.valid_examples]
        print("---------------------------------------------------------")
        print("Nearest Wordlist to candidate words: ")
        for idx, word in enumerate(valid_words):
          # skip over the original word against whom we are seeking similarity
          close_words = [reverse_dictionary[j] for j in top_k[idx]
                         if j != self.valid_examples[idx]]
          print("'%s':\t\t%s"% (word, close_words))
          
    print("=================")
    print("Training Completed")
    feed_dict = {}
    self.embeds = s.run( self.embeds_op, feed_dict=feed_dict)

In [0]:
def plot_cbow_words(self):
  tsne = TSNE(perplexity=self.perplexity, n_components=2,
              init='pca', n_iter=self.n_iter)
  embeds = tsne.fit_transform(self.embeds[1:self.n_points+1, :])
  words = [reverse_dictionary[idx] for idx in range(1, self.n_points + 1)]

  pylab.figure(figsize=(15,15))  # in inches
  for i, label in enumerate(words):
    x, y = embeds[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
  pylab.show()

In [0]:
class CbowModel():
  # Data Extraction
  word_index  = 0
  # Learning Hyperparameters
  l2_reg            = 0.01
  dropout        = 0.95
  lr                     = 1.0
  # Dimensional Hyperparameters
  cbow_win     = 1
  batch_size   = 128
  embedding_size = 128
  # Training Hyperparameters
  # E(-ve words): estimator are -ve samples drawn from another distribution 
  n_samples   = 64 
  n_avg            = 2000
  n_validate   = 10*n_avg
  n_steps        = 10*n_validate
  # Validation: Distributed Dimensions (Embedding) Validation
  valid_size     = 16 # Random words on which we evaluate similarity
  valid_win      = 100 # High frequency words from which we sample validation
  valid_top_k = 8
  # TSNE and Display
  n_points       = 400
  n_iter            = 5000
  perplexity    = 30
  # Class Methods
 g et_data = gen_cbow_batch
  define_state = define_state_cbow
  define_computation = define_computation_cbow
  run_epochs = run_epochs_cbow
  plot_words = plot_cbow_words

  def __init__(self):
    self.graph = tf.Graph()
    # Skip 'UNK' word
    rnd_word_idxs = random.sample(range(1, self.valid_win+1),self.valid_size)
    self.valid_examples = np.array(rnd_word_idxs, dtype=np.int32)
    
cbowMod = CbowModel()    

In [0]:
cbowMod.define_state()
cbowMod.define_computation()
cbowMod.run_epochs()

In [0]:
cbowMod.plot_words()